https://yandex.ru/dev/direct/doc/dg/concepts/overview.html

https://yandex.ru/dev/direct/doc/dg-v4/concepts/About.html

https://yandex.ru/dev/id/doc/dg/oauth/concepts/about.html

https://yandex.ru/dev/id/doc/dg/index.html

https://docs.python.org/3/library/json.html

In [1]:
# -*- coding: utf-8 -*-
import requests
import json
import pandas as pd

from flask import Flask, request, jsonify, redirect
from urllib.parse import urlencode

In [2]:
# параметры приложения
client_id1 = '2ea405b379c7474e8bf9e1a3bf31ee2a'
client_secret1 = '86dcb9f2c8664399b553bc98e44da1df'

# я.директ учетная запись 1
login1 = 'sfedyusnin@yandex.ru'
token1 = 'y0_AQAAAABihNGYAAhQ1wAAAADLKvrhX8L69sCOQii3e7pjpQ0tsAU4Ha4'

# я.директ учетная запись 2
login2 = 'test150822-1@yandex.ru' 
token2 = 'y0_AQAAAABj2FshAAhQ1wAAAADLnEO5KjPw9q9tRUuL9DlWVNgRkdHQDEw'

In [3]:
class YandexDirectEcomru:
    def __init__(self, login, token, sandbox=False):
        self.login = login
        self.token = token
        urls = ['https://api.direct.yandex.ru/v4/json/',
                'https://api.direct.yandex.ru/live/v4/json/',
                'https://api.direct.yandex.com/json/v5/']
        
        sandbox_urls = ['https://api-sandbox.direct.yandex.ru/v4/json/',
                        'https://api-sandbox.direct.yandex.ru/live/v4/json/',
                        'https://api-sandbox.direct.yandex.com/json/v5/']
        
        self.head = {"Authorization": 'Bearer'+' '+self.token,
                     "Accept-Language": "ru",
                     "Client-Login": self.login,
                     "Content-Type": "application/json; charset=utf-8"
                    }
        
        if sandbox is True:
            self.urls = sandbox_urls
        else:
            self.urls = urls

    
    def get_campaigns(self, criteries={}):
        """
        Возвращает параметры кампаний, отвечающих заданным критериям.
        Структура описания критериев:
        https://yandex.ru/dev/direct/doc/ref-v5/campaigns/get.html#input__CampaignsSelectionCriteria
        """
        method = 'campaigns'
        body = {"method": "get",
                "params": {"SelectionCriteria": criteries,
                           "FieldNames": ["Id", "Name"]
                          }
               }
        
        response = requests.post(self.urls[2]+method, headers=self.head, data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text

    def create_new_wordstat_report(self, phrases, regions=[], lim=10):
        """
        Запускает на сервере формирование отчета о статистике поисковых запросов,
        возвращает идентификатор будущего отчета.
        """
        body = {"method": "CreateNewWordstatReport",
                "param": {"Phrases": phrases,
                          "GeoID": regions
                         },
#                 "locale": "ru",
                "token": self.token
               }
        
        if len(phrases) <= lim:
            response = requests.post(self.urls[1], data=json.dumps(body))
            if response.status_code == 200:
                return response.json()
            else:
                return response.text
        else:
            print('Превышен лимит фраз в одном запросе')

    def get_wordstat_report_list(self):
        """
        Возвращает список сформированных и формируемых отчетов о статистике поисковых запросов
        """
        body = {"method": "GetWordstatReportList",
                "token": self.token}
        response = requests.post(self.urls[1], data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text
    
    def get_wordstat_report(self, report_id):
        """
        Возвращает отчет о статистике поисковых запросов
        """
        body = {"method": "GetWordstatReport",
                "param": int(report_id),
                "token": self.token
               }
        response = requests.post(self.urls[1], data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text
        
    def delete_wordstat_report(self, report_id):
        """
        Удаляет отчет о статистике поисковых запросов
        """
        body = {"method": "DeleteWordstatReport",
                "param": int(report_id),
                "token": self.token
               }
        response = requests.post(self.urls[1], data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text
        
    def create_new_forecast(self, phrases, regions=[], currency='RUB', auc_bids='No'):
        """
        Запускает на сервере формирование прогноза показов, кликов и затрат.
        Возможные значения currency: RUB, CHF, EUR, KZT, TRY, UAH, USD, BYN.
        """
        body = {"method": "CreateNewForecast",
                "param": {"Phrases": phrases,
                          "Categories": [],
                          "GeoID": regions,
                          "Currency": currency,
                          "AuctionBids": auc_bids
                         },
                "token": self.token
               }
        response = requests.post(self.urls[1], data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text
        
    def get_forecast_list(self):
        """
        Возвращает список сформированных и формируемых отчетов о прогнозируемом количестве
        показов и кликов, затратах на кампанию
        """
        body = {"method": "GetForecastList",
                "token": self.token}
        response = requests.post(self.urls[1], data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text
        
    def get_forecast(self, forecast_id):
        """
        Возвращает сформированный прогноз показов, кликов и затрат по его идентификатору
        """
        body = {"method": "GetForecast",
                "param": int(forecast_id),
                "token": self.token
               }
        response = requests.post(self.urls[1], data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text 
      
    def delete_forecast_report(self, forecast_id):
        """
        Удаляет отчет о прогнозируемом количестве показов и кликов, затратах на кампанию
        """
        body = {"method": "DeleteForecastReport",
                "param": int(forecast_id),
                "token": self.token
               }
        response = requests.post(self.urls[1], data=json.dumps(body))
        if response.status_code == 200:
            return response.json()
        else:
            return response.text
        

In [4]:
direct = YandexDirectEcomru(login=login1, token=token1, sandbox=True)

In [6]:
print(direct.get_campaigns.__doc__)
direct.get_campaigns()


        Возвращает параметры кампаний, отвечающих заданным критериям.
        Структура описания критериев:
        https://yandex.ru/dev/direct/doc/ref-v5/campaigns/get.html#input__CampaignsSelectionCriteria
        


{'result': {'Campaigns': [{'Name': 'Test API Sandbox campaign 1',
    'Id': 440401},
   {'Name': 'Test API Sandbox campaign 2', 'Id': 440402},
   {'Name': 'Test API Sandbox campaign 3', 'Id': 440403}]}}

In [7]:
direct.create_new_wordstat_report(phrases=['hamster'])

{'data': 7037263}

In [8]:
direct.get_wordstat_report_list()

{'data': [{'ReportID': 7037192, 'StatusReport': 'Done'},
  {'ReportID': 7037196, 'StatusReport': 'Done'},
  {'StatusReport': 'Done', 'ReportID': 7037204},
  {'ReportID': 7037208, 'StatusReport': 'Done'},
  {'ReportID': 7037227, 'StatusReport': 'Done'},
  {'ReportID': 7037230, 'StatusReport': 'Done'},
  {'StatusReport': 'Done', 'ReportID': 7037233},
  {'ReportID': 7037236, 'StatusReport': 'Done'},
  {'StatusReport': 'Done', 'ReportID': 7037237},
  {'StatusReport': 'Done', 'ReportID': 7037256},
  {'ReportID': 7037263, 'StatusReport': 'Done'}]}

In [9]:
direct.get_wordstat_report(report_id=7037233)

{'data': [{'Phrase': 'hamster',
   'SearchedWith': [{'Shows': 80728, 'Phrase': 'hamster'},
    {'Phrase': "hamster's", 'Shows': 80726},
    {'Shows': 10122, 'Phrase': 'hamster porno'},
    {'Phrase': 'x hamster', 'Shows': 9754},
    {'Shows': 9754, 'Phrase': "x hamster's"},
    {'Shows': 8533, 'Phrase': 'hamster porn'},
    {'Phrase': 'hamster порно', 'Shows': 4892},
    {'Phrase': 'hamster xxx', 'Shows': 3347},
    {'Phrase': "xxx hamster's", 'Shows': 3347},
    {'Shows': 2557, 'Phrase': 'hamster com'},
    {'Shows': 2557, 'Phrase': "hamster's com"},
    {'Phrase': 'x hamster porno', 'Shows': 2510},
    {'Phrase': 'hamster türk', 'Shows': 2386},
    {'Phrase': 'hamster porno türk', 'Shows': 2357},
    {'Phrase': 'x hamster türk', 'Shows': 2356},
    {'Phrase': 'x hamster türk porno', 'Shows': 2344},
    {'Phrase': 'hamster live', 'Shows': 2177},
    {'Shows': 2015, 'Phrase': 'hamster sex'},
    {'Shows': 1947, 'Phrase': 'ru hamster'},
    {'Shows': 1873, 'Phrase': 'hamster video'},
  

In [10]:
direct.delete_wordstat_report(report_id=7037192)

{'data': 1}

In [11]:
direct.create_new_forecast(phrases=['hamster'])

{'data': 7037265}

In [12]:
direct.get_forecast_list()

{'data': [{'ForecastID': 7037252, 'StatusForecast': 'Pending'},
  {'ForecastID': 7037257, 'StatusForecast': 'Pending'},
  {'ForecastID': 7037265, 'StatusForecast': 'Pending'}]}

In [13]:
direct.get_forecast(forecast_id=7037252)

{'error_code': 74,
 'error_str': 'Forecast report is being generated',
 'error_detail': ''}

In [14]:
direct.delete_forecast_report(forecast_id=7037265)

{'data': 1}

In [15]:
direct.get_forecast_list()

{'data': [{'StatusForecast': 'Pending', 'ForecastID': 7037252},
  {'ForecastID': 7037257, 'StatusForecast': 'Pending'}]}

In [16]:
direct2 = YandexDirectEcomru(login=login2, token=token2, sandbox=True)

In [17]:
direct2.get_campaigns()

{'result': {'Campaigns': [{'Name': 'Test API Sandbox campaign 1',
    'Id': 440596},
   {'Id': 440597, 'Name': 'Test API Sandbox campaign 2'},
   {'Id': 440598, 'Name': 'Test API Sandbox campaign 3'}]}}

In [18]:
ph = (u'пух').encode().decode('utf8')
ph

'пух'

In [19]:
# ph = ('пух -винни').encode('UTF8')

In [20]:
direct2.create_new_wordstat_report(phrases=[ph])

{'error_str': 'Invalid request',
 'error_detail': 'Request encoding is not UTF8',
 'error_code': 501}